# Finding the cultural capital of Europe

We are now ready for the big moment of finding which city is the cultural capital of Europe based on our criteria.

## Perform Analysis on all the cultural features across all cities

As we have seen in the previous chapter it's possible for us to perform a spaital join between two sets of geometries  for a given spatial predicate. This required the following three Steps:

### Data preperation for doing Spatial Join via Spatial Spark

* each feature in each set having a unique id
* creating a dataframe with a tuple containing feature's unique id and it's geometry as a WKT string from each dataset
* passing the Spark dataframe to the `BroadcastSpatialJoin` function provided by SpatialSpark 

The result of the `BroadcastSpatialJoin` is a list of tuples with the ID's of features that meet our join criteria. So essentially a join table for the two datasets. 

### Appending Properties back to the join results

We then have to take these tuples and append our properties from each dataset namely for the cultural POIs labels and for the cities the city names and the population. 

### Doing analysis on join output

We must then count how many of each type of cultural locations are there for each city. When we know how many museums, galleries, theaters and artworks there are in each city we can then calculated the weighted score for each location and then sum them for each city. 

So lets get started!

In [ ]:
# %load './code/helpers/imports.py'
import os.path, json, io, pandas
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (54, 60)

# for exponential back down when calling TurboOverdrive API
from retrying import retry 

# resuse as func.coalace for example
import pyspark.sql.functions as func 
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DecimalType

# for loading boundaries Data
from geopandas import GeoDataFrame 
# for creating geospatial data
from shapely.geometry import Point, Polygon, shape 
# for creating and parsing geospatial data in WKT and WKB format
from shapely import wkb, wkt 
# for accessing OpenStreetMap API
import overpy 

try:
    sc
except NameError:
    import pyspark
    sc = pyspark.SparkContext('local[*]')
    sqlContext = pyspark.sql.SQLContext(sc)

In [ ]:
# features = pandas.read_json(POIS_PATH)
# features['properties']

In [ ]:
# %load './code/helpers/load_boundaries_and_pois.py'
OVERPASS_API          = overpy.Overpass()
BASE_DIR              = os.path.join(os.path.abspath('.'), 'work-flow')
URBAN_BOUNDARIES_FILE = "06_Europe_Cities_Boundaries_with_Labels_Population.geo.json"

# Paths to base datasets that we are using:
URBAN_BOUNDARIES_PATH = os.path.join(BASE_DIR,URBAN_BOUNDARIES_FILE)
POIS_PATH             = os.path.join(BASE_DIR, "pois.json")

# Prepare data making sure that the WKT column exists
try:
    geo_df
except NameError:
    geo_df = GeoDataFrame.from_file(URBAN_BOUNDARIES_PATH)
    # Add a WKT column for use later
    geo_df['wkt'] = pandas.Series(
        map(lambda geom: str(geom.to_wkt()), geo_df['geometry']),
        index=geo_df.index, dtype='string')

# Create a Boundries Spark Dataframe from GeoPandas 
try:
    boundaries_from_pd
except NameError:
    boundaries_from_pd = sqlContext.createDataFrame(geo_df)
    boundaries_from_pd.registerTempTable("boundaries")

# Create POI Spark Dataframe from GeoJSON Features data 
# where each feature is in it's own row
try:
    pois_df
except NameError:
    pois_df = sqlContext.read.json(POIS_PATH)
    pois_df = pois_df.toPandas()
    def toWktColumn(coords):
        return (Point(coords).wkt)

    pois_df['wkt'] = pandas.Series(
        map(lambda geom: toWktColumn(geom.coordinates), pois_df['geometry']),
        index=pois_df.index, dtype='string')

    pois_df = sqlContext.createDataFrame(pois_df)

# Unique IDs
As we saw in the introduction to Spatial Spark chapter, each record in our dataset needs to have unique ID in order to allow us to get the tuple matching our Spatial predicates. If you missed that see the previous chapter titled: _Spatial Spark_. 

So we will begin by doing the following:

1. create a dataframe from a tuple with a unique id and WKT Geometry for the Cultural Points of Interests (POIs)
2. create a dataframe from a tuple with a unique id and WKT Geometry for the City Boundaries

This will then be passed to Spatial Sparks API to obtain a Spatial Join. To create the unique IDs we will use the function `monotonicallyIncreasingId` provided by PySpark.

In [ ]:
from pyspark.sql.functions import monotonicallyIncreasingId

# create dataframe with (id, geometry) for POIs
# 1. Add and ID Column to POIs

pois_df           = pois_df.withColumn("id", monotonicallyIncreasingId())
pois_tuple_id_wkt = pois_df.select(pois_df['id'], pois_df['wkt'])

pois_tuple_id_wkt.show()
pois_tuple_id_wkt.printSchema()
print pois_tuple_id_wkt.count()

In [ ]:
# create dataframe with (id, geometry-as-WKT) for boundaries
# 1. Add and ID Column to boundaries

boundaries_from_pd      = boundaries_from_pd.withColumn("id", monotonicallyIncreasingId())
boundaries_tuple_id_wkt = boundaries_from_pd.select(boundaries_from_pd['id'], boundaries_from_pd['wkt'])

boundaries_tuple_id_wkt.printSchema()
print boundaries_from_pd.count()
boundaries_tuple_id_wkt.show()
wkt.loads(boundaries_tuple_id_wkt.take(7)[6].wkt)

In [ ]:
spatialspark = sc._jvm.spatialspark
SpatialOperator      = spatialspark.operator.SpatialOperator 
BroadcastSpatialJoin = spatialspark.join.BroadcastSpatialJoin
from ast import literal_eval as make_tuple # used to decode data from java

joinPoiBdryRDD = BroadcastSpatialJoin.apply(sc._jsc, 
                                            pois_tuple_id_wkt._jdf, 
                                            boundaries_tuple_id_wkt._jdf, 
                                            SpatialOperator.Within(), 
                                            0.0)

In [ ]:
print joinPoiBdryRDD.count()

joinResults = map(lambda result: make_tuple(result.toString()), joinPoiBdryRDD.collect())


In [ ]:
print pois_tuple_id_wkt.count()
print boundaries_tuple_id_wkt.count()
print joinResults[695]

In [ ]:
# make the results a DF
rddResult = sc.parallelize(joinResults)
df = sqlContext.createDataFrame(rddResult, ["poi_id", "boundry_id"])
df.printSchema()

In [ ]:
# Do a join with poi df and bdry df
df_with_pois = df.join(pois_df, df['poi_id'] == pois_df['id']).select(
    df['poi_id'],
    df['boundry_id'],
    pois_df['properties'].alias("poi_properties"),    
    pois_df['wkt'].alias("poi_wkt")
)

In [ ]:
df_with_pois.columns

In [ ]:
df_with_pois_bdrys = df_with_pois.join(
        boundaries_from_pd, df_with_pois['boundry_id'] == boundaries_from_pd['id']
    ).select(
        df_with_pois['poi_id'],
        df_with_pois['boundry_id'],
        df_with_pois['poi_properties'],
        df_with_pois['poi_wkt'],
        boundaries_from_pd['wkt'].alias("boundry_wkt"),
        boundaries_from_pd['NAMEASCII'].alias("city_name"),
        boundaries_from_pd['POPEU2013'].alias("population")
    )

In [ ]:
df_with_pois_bdrys.columns

In [ ]:
df_with_pois_bdrys.cache()
df_with_pois_bdrys.count()

In [ ]:
df_with_pois_bdrys.show(3)
# we have now

In [ ]:
rec = df_with_pois_bdrys.take(1)[0]

In [ ]:
print rec.poi_properties.amenity or rec.poi_properties.tourism

In [ ]:
# For the first step we add columns for each tag. In this case we have amenity and tourism

df_with_pois_bdrys = df_with_pois_bdrys.withColumn(
        'tourism', df_with_pois_bdrys['poi_properties']['tourism']
    ).withColumn(
        'amenity', df_with_pois_bdrys['poi_properties']['amenity'])

df_with_pois_bdrys.columns

We next coalesce the two columns into a single column called `location_type` and then group our data by all the fields we need in this case: `city_name`, `population`, `location_type` and then perform a count.

In [ ]:
df = df_with_pois_bdrys.select('*', func.coalesce(
        df_with_pois_bdrys['poi_properties']['tourism'], 
        df_with_pois_bdrys['poi_properties']['amenity']
    ).alias("location_type")).groupby('boundry_id', 
                                      'city_name', 
                                      'location_type', 
                                      'population').count()

In [ ]:
df.show(10)

This gives us the **count per location_type per city**. Next we need the weighted score for each location type and then perform a grouped sum of the **score** for each city. 

In order to perform the grouped sum we will create a UDF (user defined function) that given a `location_type`, `count` of the location and `population` for the city in question will return a single score value weighted by the location type and normalised by the population.

In [ ]:
# Create an UDF for a column that calculates the score per record

def get_cultural_score(location_type, count, population):
    cultural_weight_lookup = { 
        u'museum':      3.0,
        u'arts_centre': 2.0,
        u'theatre':     2.0,
        u'gallery':     2.0,
        u'artwork':     0.0  # try modifying the weights as an exercise
    }

    wgt = cultural_weight_lookup.get(location_type, 0.0)

    return round(float((wgt* float(count) * 100000)/float(population)), 2)

sqlContext.registerFunction("get_cultural_score", get_cultural_score, FloatType())7

# score_udf = func.udf(get_cultural_score, FloatType())

In [ ]:
# score_df = df.select(df.boundry_id, 
#           df.city_name, 
#           df.population,
#           df.location_type,
#           df.count,
#           score_udf(df.location_type, df.count, df.population).alias('cultural_score')
#          )

In [ ]:
df.registerTempTable("cultural_score")

score_df = sqlContext.sql(
    "SELECT boundry_id, \
        city_name, \
        location_type, \
        population, \
        count, get_cultural_score(location_type, count, population) as score \
    FROM cultural_score")

In [ ]:
score_df.sort(score_df['city_name'].asc()).show()

In [ ]:
df = (score_df
        .groupBy("boundry_id", "city_name", "population")
        .agg(func.round(func.sum(score_df['score']), 2).alias('final_score'))
        .sort("city_name"))

In [ ]:
df.show(4)

In [ ]:
df.collect()
(df
 .select(df['city_name'], df['population'], df['final_score'])
 .sort(df['final_score'])
 .show(df.count()))

# Visualizing the data

Let map the data using the Folium package that allows embedding Leaflet Maps inside ipython notebooks.


In [ ]:
pd_df = df_with_pois_bdrys.toPandas()
pd_df.head(2)
df_with_pois_bdrys.columns

In [ ]:
pd_df = pd_df[['boundry_id', 'boundry_wkt', 'city_name', 'population']]
pd_df = pd_df.drop_duplicates()
pd_df.columns

In [ ]:
# Geometry list
geometry = [wkt.loads(boundry_wkt) for boundry_wkt in pd_df.boundry_wkt]

In [ ]:
# create a GeodataFrame
geodf = GeoDataFrame(pd_df, geometry=geometry)
geodf.columns

In [ ]:
scores = pd

In [ ]:
scores.head()
scores_merged_df = pandas.DataFrame.merge(geodf, scores, on='boundry_id')

In [ ]:
scores_merged_df.head(1)
geo_scores_merged = GeoDataFrame(scores_merged_df)

In [ ]:
geo_scores_merged.columns

In [ ]:
import folium
map_osm = folium.Map(location=[47.19094, 11.98566], 
    tiles='Mapbox Bright',
    zoom_start=6)

map_osm.choropleth(geo_str=geo_scores_merged.to_json(),
              data=geo_scores_merged,
              columns=['city_name_x', 'final_score'],
              fill_color='RdBu',
              key_on='properties.city_name_x')
map_osm